In [36]:
import pygsheets
import jieba
import jieba.analyse
from gensim.models.fasttext import FastText
import pandas as pd
import numpy as np
#model = FastText.load_fasttext_format('cc.zh.300.bin')   #中文語料庫
gc = pygsheets.authorize(service_file='autoreply.json')  #取得憑證
url='https://docs.google.com/spreadsheets/d/1S8Bjn85zvB12M3-A5Hk1wnWJDfMO5PkzPsQMZT5dE4c/edit?usp=sharing'
sht = gc.open_by_url(url)   #連結google sheet


wks_list = sht.worksheets()    #所有sheet
wks=sht[0]                     #QA sheet
wks1=sht[1]                    #未知問題

keyword_list=wks.get_col(3,include_tailing_empty=False)[1:]  #去掉第一欄


def gc2df(wks):
    data = wks.get_all_values(include_tailing_empty=False,include_tailing_empty_rows=False)
    headers = data.pop(0)
    df = pd.DataFrame(data, columns=headers)
    return df 

def generate_compareList(keyword_list):
    compare_list=[]
    for index,keyword in enumerate(keyword_list) :
        compare_list.append((index,keyword.split(',')))
    return compare_list

def find_answer_ID(compare_list,keywords):
    answer=[]
    for item in compare_list :
        if cmp_similarity(item[1],keywords) > 0.55 :
            keywords=item[1]       
        answer.append([set(keywords)&set(item[1]),item[0]])
    if len(max(answer)[0]) > 0 :
        return max(answer)[1]
    else :
        return False
    
def keyword_string(keywords):
    string=""
    for keyword in keywords:
        string=string+keyword+","
    return string

def keyword_insert(index,question,content):
    wks1.update_value((index+1,1),index)
    wks1.update_value((index+1,2),question)
    wks1.update_value((index+1,3),content)
    return 1

def cmp_similarity(list2,list1):
    ans=[]
    for text in list2 :
        for keyword in list1:
            ans.append((model.wv.similarity(keyword,text),[keyword,text]))
    
    return max(ans)[0] 

In [29]:
if __name__ == '__main__' :
    jieba.load_userdict('Corpus.txt')
    question=input("請輸入你的問題")
    keywords=jieba.analyse.extract_tags(question,topK=3,withWeight=False)
    compare_list=generate_compareList(keyword_list)
    if find_answer_ID(compare_list,keywords) == False :
        print("目前找不到答案")
        index=len(wks1.get_col(1,include_tailing_empty=False))
        content=keyword_string(keywords)
        keyword_insert(index,question,content)
    else :
        answer_ID=find_answer_ID(compare_list,keywords)
        print(wks.get_value((answer_ID+2,4)))

In [37]:
#將google sheet 表單轉為 np.arry放進決策樹模型中訓練

df=gc2df(wks)
print(df)
ls=[]
for keyword in df['keywords'] :
    ls.append(keyword.split(','))
x=[]
print(ls)
for i in ls :
    for j in i :
        x.append(j)
print(x)

label_x=np.zeros((17,17))    #生成 0 矩陣

df=pd.DataFrame(data=label_x,columns=x,index=x)

for i in df.index :
    for j in df.columns :
        if i == j :
            df.loc[i,j]=1
            
print(df.values)
target=np.array([0,0,0,0,1,1,1,2,3,4,5,6,7,7,7,7,8])   #目標答案
print(type(target))
print(target)
x=df.values
y=target

  Question_ID    Question     keywords         Answer
0           0     今天幾點開門?  開門,營業,關門,結束   早上九點營業晚上十點關門
1           1      可以刷卡嘛?     刷卡,付現,現金    可以刷卡也可以付現金唷
2           2      可以訂位嘛?           訂位        不提供訂位服務
3           3     可以帶寵物嘛?           寵物         不行攜帶寵物
4           4       有素食嘛?           素食          有提供素食
5           5      可以外送嗎?           外送        有提供外送服務
6           6  有LinePay嗎?      LinePay  可以使用LinePay付款
7           7     最多人點甚麼?  好吃,推薦,食物,招牌     本店的鴨腿飯非常好吃
8           8       有菜單嗎?           菜單   臉書粉專有菜單可以參考唷
[['開門', '營業', '關門', '結束'], ['刷卡', '付現', '現金'], ['訂位'], ['寵物'], ['素食'], ['外送'], ['LinePay'], ['好吃', '推薦', '食物', '招牌'], ['菜單']]
['開門', '營業', '關門', '結束', '刷卡', '付現', '現金', '訂位', '寵物', '素食', '外送', 'LinePay', '好吃', '推薦', '食物', '招牌', '菜單']
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0.

In [28]:
#決策樹預測

import numpy as np
from sklearn import datasets        #引入sklearn中的資料集
from sklearn.tree import DecisionTreeClassifier  #引入sklearn中的決策樹分類器
from matplotlib.colors import ListedColormap     #引入matplotlib中的ListedColormap能夠畫出自己的Colormap
import matplotlib.pyplot as plt    #引入matplotib 畫圖
from sklearn.model_selection import train_test_split   #用來隨機劃分訓練集和資料集

X_train, X_test, y_train, y_test = train_test_split(
    x,y, test_size=0.3, random_state=0)


tree = DecisionTreeClassifier(criterion='entropy',
                              max_depth=3, random_state=0)

# tree.fit(X_train,y_train)
tree.fit(x,y)

expected=target
predicted=tree.predict(x)

print(expected)
print(predicted)

[0 0 0 0 1 1 1 2 3 4 5 6 7 7 7 7 8]
[0 0 0 0 0 0 0 0 0 4 5 0 0 0 0 0 8]
